**Generating access token**

This Python code interacts with the Spotify API to request an access token using the Client Credentials flow. It starts by importing the necessary libraries, requests for making HTTP requests and base64 for encoding. The CLIENT_ID and CLIENT_SECRET are combined and Base64 encoded for secure authorization. This encoded string is then used in an HTTP POST request to Spotify's token endpoint, along with a data payload specifying the grant_type as "client_credentials".

In [ ]:
import requests
import base64

CLIENT_ID = 'f{Your client id}'
CLIENT_SECRET = 'f{Your client secret}'


client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())


token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()

Access token obtained successfully.


**Defining Features**

This Python function, get_trending_playlist_data, retrieves detailed information about the tracks in a given Spotify playlist using the Spotipy library. It takes a playlist ID and an access token as inputs and accesses Spotify's API to fetch the track details, such as track name, artists, album name, and album ID. Additionally, it retrieves audio features (like danceability, energy, and tempo), popularity, release date, and other metadata for each track. The function compiles this information into a list of dictionaries, which is then converted into a pandas DataFrame, making the data easily accessible for further analysis or export.

In [ ]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

def get_trending_playlist_data(playlist_id, access_token):

    sp = spotipy.Spotify(auth=access_token)


    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')


    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']


        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None


        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None


        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None


        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,

        }

        music_data.append(track_data)


    df = pd.DataFrame(music_data)

    return df

**Retreive Music Data from Playlist**


In [ ]:
playlist_id = 'https://open.spotify.com/playlist/5cfIZWLbFP5bn9C7UfxLbI?si=2bf4abeef14d40f0'

music_df = get_trending_playlist_data(playlist_id, access_token)

print(music_df)

                                           Track Name  \
0                                    Main Agar Kahoon   
1                                         Khuda Jaane   
2                                    Ye Tune Kya Kiya   
3                                           Tum Se Hi   
4                                             Zara Sa   
..                                                ...   
63                                            O Maahi   
64                                 Bol Na Halke Halke   
65  Ve Kamleya (Sufi Version) (From "Rocky Aur Ran...   
66                              Humko Humise Chura Lo   
67                                           Tere Bin   

                                              Artists  \
0                          Sonu Nigam, Shreya Ghoshal   
1   Vishal-Shekhar, KK, Shilpa Rao, Anvita Dutt Gu...   
2                                Pritam, Javed Bashir   
3                               Pritam, Mohit Chauhan   
4                             

In [ ]:
print(music_df.isnull().sum())

Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64


**Importing the Dependencies**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

data = music_df

**Defining Weight of tracks**

The function calculate_weighted_popularity calculates a weight for a track's popularity based on its release date. It first converts the provided release date (in the format 'YYYY-MM-DD') into a Python datetime object. Then, it calculates the time span between the release date and the current date. The weight is inversely proportional to the time span, meaning that more recent releases receive a higher weight. This weight can be used to adjust a track's popularity score, giving newer releases more influence in the ranking or analysis.

In [ ]:
def calculate_weighted_popularity(release_date):

    release_date = datetime.strptime(release_date, '%Y-%m-%d')


    time_span = datetime.now() - release_date

    weight = 1 / (time_span.days + 1)
    return weight

**Normalising Features**

This code snippet uses MinMaxScaler from the sklearn.preprocessing module to normalize a set of music feature values. It first selects specific columns from a DataFrame music_df, which represent various audio features such as danceability, energy, loudness, tempo, and others. These features are then extracted as a NumPy array using the .values method. The MinMaxScaler scales each feature to a range between 0 and 1 by fitting it to the data and then transforming it. The result, music_features_scaled, contains the normalized feature values, making the data more suitable for machine learning algorithms.

In [ ]:
scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key',
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

**Generating music recommendations**

The content_based_recommendations function generates music recommendations based on the features of a given input song using content-based filtering. It takes the input song's name and the number of recommendations to return. If the input song is not found in the dataset (music_df), an error message is displayed. The function retrieves the index of the input song and calculates the cosine similarity between its audio features and those of other songs in the dataset. Based on these similarity scores, it identifies the most similar songs and returns a DataFrame containing the recommended tracks, along with their artist, album, release date, and popularity.

In [ ]:
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]


    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

    return content_based_recommendations

 **Combining content-based filtering and popularity weighting**

 The hybrid_recommendations function combines content-based filtering and popularity weighting to provide song recommendations. It starts by checking if the input song exists in the dataset (music_df). If it does, content-based recommendations are generated based on the input song’s audio features. The function then retrieves the popularity score of the input song and adjusts it using a weighted popularity score based on the song's release date. The input song is added to the recommendations, and all tracks are sorted by popularity. Finally, the input song itself is excluded from the list of recommendations, and the top tracks are returned. This hybrid approach balances feature similarity and popularity.

In [ ]:
import pandas as pd

def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)

    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]

    weighted_popularity_score = popularity_score * calculate_weighted_popularity(
        music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0]
    )

    new_entry = pd.DataFrame({
        'Track Name': [input_song_name],
        'Artists': [music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0]],
        'Album Name': [music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0]],
        'Release Date': [music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0]],
        'Popularity': [weighted_popularity_score]
    })

    hybrid_recommendations = pd.concat([content_based_rec, new_entry], ignore_index=True)

    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)

    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]

    return hybrid_recommendations

**Generating recommendations**

In [ ]:
input_song_name = "Khuda Jaane"
recommendations = hybrid_recommendations(input_song_name, num_recommendations=5)
print(f"Hybrid recommended songs for '{input_song_name}':")
print(recommendations)

Hybrid recommended songs for 'Khuda Jaane':
                 Track Name                                           Artists  \
0          Ye Tune Kya Kiya                              Pritam, Javed Bashir   
4                  Pee Loon                             Pritam, Mohit Chauhan   
2  Tujh Mein Rab Dikhta Hai  Salim–Sulaiman, Roop Kumar Rathod, Jaideep Sahni   
1                Tu Hi Mera                        Pritam, Shafqat Amanat Ali   
3                   Hosanna        A.R. Rahman, Leon D'souza, Suzanne D'Mello   

                                          Album Name Release Date  Popularity  
0              Once Upon Ay Time In Mumbai Dobaara !   2013-07-22        73.0  
4                        Once Upon A Time In Mumbaai   2010-06-29        73.0  
2                                Rab Ne Bana Di Jodi   2008-11-06        69.0  
1      Jannat 2 (Original Motion Picture Soundtrack)   2012-03-30        65.0  
3  Ekk Deewana Tha (Original Motion Picture Sound...   2012-01-04    